In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# Combining DataFrames

In [2]:
ser1 = Series([2, np.nan, 4, np.nan, 6, np.nan], index = list('QRSTUV'))
ser1

Q    2.0
R    NaN
S    4.0
T    NaN
U    6.0
V    NaN
dtype: float64

In [3]:
ser2 = Series(np.arange(len(ser1)), dtype = np.float64, index = list('QRSTUV')) 
ser2

Q    0.0
R    1.0
S    2.0
T    3.0
U    4.0
V    5.0
dtype: float64

#### Two different ways to make a series based on series1 while using series2 to replace its null values.

In [4]:
np.where(pd.isnull(ser1), ser2, ser1) 

array([ 2.,  1.,  4.,  3.,  6.,  5.])

In [5]:
Series(np.where(pd.isnull(ser1), ser2, ser1), index = ser1.index) 

Q    2.0
R    1.0
S    4.0
T    3.0
U    6.0
V    5.0
dtype: float64

In [6]:
ser1.combine_first(ser2) 

Q    2.0
R    1.0
S    4.0
T    3.0
U    6.0
V    5.0
dtype: float64

#### Two different ways to make a dataframe based on dframe1 while using dframe2 to replace its null values.

In [7]:
nan = np.nan

dframe_odds = DataFrame({'X':[1., nan, 3., nan], 
                         'Y':[nan, 5., nan, 7.], 
                         'Z':[nan, 9., nan, 11.]})

In [8]:
dframe_evens = DataFrame({'X':[2., 4., nan, 6., 8.], 
                         'Y':[nan, 10., 12., 14., 16.]}) 

In [9]:
dframe_odds

,X,Y,Z
0,1.0,NaN,NaN
1,NaN,5.0,9.0
2,3.0,NaN,NaN
3,NaN,7.0,11.0


In [10]:
dframe_evens

,X,Y
0,2.0,NaN
1,4.0,10.0
2,NaN,12.0
3,6.0,14.0
4,8.0,16.0


In [11]:
dframe_odds.combine_first(dframe_evens)  

,X,Y,Z
0,1.0,NaN,NaN
1,4.0,5.0,9.0
2,3.0,12.0,NaN
3,6.0,7.0,11.0
4,8.0,16.0,NaN


# Reshaping

In [12]:
dframe1 = DataFrame(np.arange(8).reshape(2, 4)
                    , index = pd.Index(['LA', 'SF'], name = 'city')
                   , columns = pd.Index(['A', 'B', 'C', 'D'], name = 'letter'))
#We use pd.Index because it makes it possible to set the name at the same time
dframe1

letter,A,B,C,D
city,,,,
LA,0,1,2,3
SF,4,5,6,7


In [13]:
dframe1_st = dframe1.stack()
dframe1_st

city  letter
LA    A         0
      B         1
      C         2
      D         3
SF    A         4
      B         5
      C         6
      D         7
dtype: int32

In [14]:
type(dframe1_st) 

pandas.core.series.Series

In [15]:
dframe1_st.unstack() 

letter,A,B,C,D
city,,,,
LA,0,1,2,3
SF,4,5,6,7


In [16]:
dframe1_st.unstack(level='city') 

city,LA,SF
letter,,
A,0,4
B,1,5
C,2,6
D,3,7


In [17]:
dframe1_st.unstack(level='letter') 

letter,A,B,C,D
city,,,,
LA,0,1,2,3
SF,4,5,6,7


In [18]:
ser1 = Series([0, 1, 2], index = ['Q', 'X', 'Y'])
ser2 = Series([4, 5, 6], index = ['X', 'Y', 'Z']) 

In [19]:
dframe = pd.concat([ser1, ser2], keys = ['Alpha', 'Beta'])
dframe

Alpha  Q    0
       X    1
       Y    2
Beta   X    4
       Y    5
       Z    6
dtype: int64

In [20]:
type(dframe) 

pandas.core.series.Series

In [21]:
dframe = dframe.unstack(level = 0)  
dframe

,Alpha,Beta
Q,0.0,NaN
X,1.0,4.0
Y,2.0,5.0
Z,NaN,6.0


In [22]:
dframe.stack() 
#The null values will be ignored automatically

Q  Alpha    0.0
X  Alpha    1.0
   Beta     4.0
Y  Alpha    2.0
   Beta     5.0
Z  Beta     6.0
dtype: float64

In [ ]:
dframe.stack(dropna=False) 

# Pivoting

DataFrame.pivot(index=None, columns=None, values=None)[source]
Reshape data (produce a “pivot” table) based on column values. Uses unique values from index / columns to form axes of the resulting DataFrame.


Parameters:	
index : string or object, optional
    Column name to use to make new frame’s index. If None, uses existing index.
columns : string or object
    Column name to use to make new frame’s columns
values : string or object, optional
    Column name to use for populating new frame’s values. If not specified, all remaining columns will     be used and the result will have hierarchically indexed column

In [23]:
df1 = pd.DataFrame({'sex': ['male','male','female','female','female','male'],
                       'degree': ['A', 'B', 'C', 'A', 'B', 'C'],
                       'age':[20, 21, 22, 20, 20, 21],
                       'count': [2, 2, 3, 4, 2, 4]})
df1

,age,count,degree,sex
0,20,2,A,male
1,21,2,B,male
2,22,3,C,female
3,20,4,A,female
4,20,2,B,female
5,21,4,C,male


In [24]:
df1.pivot('sex', 'degree', 'count')

degree,A,B,C
sex,,,
female,4,2,3
male,2,2,4


In [25]:
df1.pivot('sex', 'degree')

age         count      
degree   A   B   C     A  B  C
sex                           
female  20  20  22     4  2  3
male    20  21  21     2  2  4

#### Use pivot_table if there are duplicates

In [26]:
df2 = pd.DataFrame({'sex': ['male','male','female','female','female','male','male','male','female'],
                       'degree': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
                       'age':[20, 21, 22, 20, 20, 21, 21, 20, 20],
                       'count': [2, 2, 3, 4, 2, 4, 1, 2, 3]})
df2

,age,count,degree,sex
0,20,2,A,male
1,21,2,B,male
2,22,3,C,female
3,20,4,A,female
4,20,2,B,female
5,21,4,C,male
6,21,1,A,male
7,20,2,B,male
8,20,3,C,female


In [27]:
#df2.pivot(index = 'sex', columns='degree') #error
df2.pivot_table(index = 'sex', columns='degree')#, aggfunc=np.sum)

age             count          
degree     A     B     C     A    B    C
sex                                     
female  20.0  20.0  21.0   4.0  2.0  3.0
male    20.5  20.5  21.0   1.5  2.0  4.0

# Duplicates in DataFrames

In [28]:
dframe = DataFrame({'key1': ['A']*2 + ['B']*3, 'key2':[2,2,2,3,3]}) 
dframe

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3


In [29]:
dframe.duplicated() 

0    False
1     True
2    False
3    False
4     True
dtype: bool

In [30]:
dframe.drop_duplicates() 

,key1,key2
0,A,2
2,B,2
3,B,3


In [31]:
dframe.drop_duplicates(['key1']) 
#It keeps the first occurance by default

,key1,key2
0,A,2
2,B,2


In [32]:
dframe

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3


In [33]:
dframe.drop_duplicates(['key1'], keep='last') 

,key1,key2
1,A,2
4,B,3


# Mapping

In [34]:
dframe = DataFrame({'city': ['Rasht', 'Tehran', 'Bam'], 
                    'population(*1000)':[3000, 16000, 100]})
dframe

,city,population(*1000)
0,Rasht,3000
1,Tehran,16000
2,Bam,100


In [35]:
state_map = {"Rasht": "Gilan", "Tehran": "Tehran", "Bam": "Kerman"}
dframe['state'] = dframe['city'].map(state_map) 
dframe

,city,population(*1000),state
0,Rasht,3000,Gilan
1,Tehran,16000,Tehran
2,Bam,100,Kerman


# Replace

In [36]:
ser1 = Series([1,2,3,4] * 2)
ser1

0    1
1    2
2    3
3    4
4    1
5    2
6    3
7    4
dtype: int64

In [37]:
ser1.replace(1, np.nan)

0    NaN
1    2.0
2    3.0
3    4.0
4    NaN
5    2.0
6    3.0
7    4.0
dtype: float64

In [38]:
ser1.replace([1,3], np.nan) 

0    NaN
1    2.0
2    NaN
3    4.0
4    NaN
5    2.0
6    NaN
7    4.0
dtype: float64

In [39]:
ser1.replace([1,3], [100, 300]) 

0    100
1      2
2    300
3      4
4    100
5      2
6    300
7      4
dtype: int64

In [40]:
ser1.replace({1:np.nan, 2:200}) 

0      NaN
1    200.0
2      3.0
3      4.0
4      NaN
5    200.0
6      3.0
7      4.0
dtype: float64

# Renaming Index

In [41]:
dframe = DataFrame(np.arange(12).reshape(3, 4), index = ['Rasht', 'Qom', 'Tehran'],
                   columns = ['A', 'B', 'C', 'D']) 
dframe

,A,B,C,D
Rasht,0,1,2,3
Qom,4,5,6,7
Tehran,8,9,10,11


In [42]:
dframe.index.map(str.upper) 

array(['RASHT', 'QOM', 'TEHRAN'], dtype=object)

In [43]:
dframe.index = dframe.index.map(str.upper) 
dframe

,A,B,C,D
RASHT,0,1,2,3
QOM,4,5,6,7
TEHRAN,8,9,10,11


In [44]:
dframe.columns = dframe.columns.map(str.lower) 
dframe.columns

Index([u'a', u'b', u'c', u'd'], dtype='object')

In [45]:
def myMap(input):
    return input + ",,,"
dframe.columns.map(myMap) 

array(['a,,,', 'b,,,', 'c,,,', 'd,,,'], dtype=object)

In [46]:
dframe

,a,b,c,d
RASHT,0,1,2,3
QOM,4,5,6,7
TEHRAN,8,9,10,11


In [47]:
dframe.rename(index = str.title, columns = str.upper, inplace=True)
dframe

,A,B,C,D
Rasht,0,1,2,3
Qom,4,5,6,7
Tehran,8,9,10,11


In [48]:
dframe.rename(index = myMap, columns = myMap)

,"A,,,","B,,,","C,,,","D,,,"
"Rasht,,,",0,1,2,3
"Qom,,,",4,5,6,7
"Tehran,,,",8,9,10,11


In [49]:
dframe.rename(index = {"Tehran":"Tehran, capital"}, columns = {"A":"Alpha"}, inplace=True)
dframe

,Alpha,B,C,D
Rasht,0,1,2,3
Qom,4,5,6,7
"Tehran, capital",8,9,10,11


# Binning

In [50]:
years = [1990, 1991, 1992, 2008, 2012, 2015, 1987, 1969, 2013, 2008, 1999]
decade_bins = [1960, 1970, 1980, 1990, 2000, 2010, 2020]

In [51]:
decade_cats = pd.cut(years, decade_bins) 
decade_cats

[(1980, 1990], (1990, 2000], (1990, 2000], (2000, 2010], (2010, 2020], ..., (1980, 1990], (1960, 1970], (2010, 2020], (2000, 2010], (1990, 2000]]
Length: 11
Categories (6, object): [(1960, 1970] < (1970, 1980] < (1980, 1990] < (1990, 2000] < (2000, 2010] < (2010, 2020]]

In [52]:
decade_cats.categories

Index([u'(1960, 1970]', u'(1970, 1980]', u'(1980, 1990]', u'(1990, 2000]',
       u'(2000, 2010]', u'(2010, 2020]'],
      dtype='object')

In [53]:
type(decade_cats)

pandas.core.categorical.Categorical

In [55]:
# pd.value_counts(decade_cats) 
decade_cats.value_counts()

(1960, 1970]    1
(1970, 1980]    0
(1980, 1990]    2
(1990, 2000]    3
(2000, 2010]    2
(2010, 2020]    3
dtype: int64

In [61]:
pd.cut(years, bins=2, labels=['First half', 'Second half']) 

[First half, First half, First half, Second half, Second half, ..., First half, First half, Second half, Second half, Second half]
Length: 11
Categories (2, object): [First half < Second half]

# Outliers

In [62]:
dframe = DataFrame(np.random.randn(1000, 4)) 
dframe.head(n=3)  

,0,1,2,3
0,0.029016,0.608740,-1.817524,0.575563
1,2.337636,-1.997483,0.079537,1.316726
2,-1.514595,0.676170,0.037646,1.621099


In [63]:
dframe.tail() 

,0,1,2,3
995,-0.478135,1.615306,-0.220816,0.269240
996,0.729078,0.369225,-1.229269,0.047697
997,0.465196,-2.116974,-0.125885,-0.021649
998,-0.968141,-0.745725,0.778541,-0.604415
999,-0.065065,1.392635,0.956144,-1.989568


In [64]:
dframe.describe() 

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.017682,0.022759,0.004315,0.066323
std,0.984102,0.955898,0.971865,1.002161
min,-2.584098,-3.332086,-2.978282,-3.717075
25%,-0.656768,-0.655624,-0.669073,-0.601413
50%,-0.009375,0.000355,0.025975,0.027492
75%,0.662285,0.665312,0.647199,0.710819
max,2.892183,3.300581,2.897228,3.125358


In [66]:
col = dframe[0] 
col.head() 

0    0.029016
1    2.337636
2   -1.514595
3   -1.036699
4    1.017568
Name: 0, dtype: float64

In [72]:
col[np.abs(col) > 2.5] 

11     2.651683
13    -2.531246
40     2.511561
208    2.892183
379   -2.502356
427    2.785876
450    2.587258
631    2.550481
813   -2.584098
855    2.774736
Name: 0, dtype: float64

In [73]:
dframe[(np.abs(dframe) > 3).any(axis = 1)]  

,0,1,2,3
62,0.946096,3.300581,-0.707454,-1.568521
172,1.078302,-3.173647,2.455854,-0.100852
329,-0.789753,-0.244968,0.425550,3.125358
835,-0.081801,-3.332086,0.380747,0.470035
952,0.241610,-3.112995,-1.222211,-0.280264
968,-0.364401,-0.771208,-0.577865,-3.717075


In [74]:
dframe[np.abs(dframe) > 3] = 3 * np.sign(dframe) 

In [75]:
dframe.describe() 

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.017682,0.023077,0.004315,0.066915
std,0.984102,0.952870,0.971865,0.999329
min,-2.584098,-3.000000,-2.978282,-3.000000
25%,-0.656768,-0.655624,-0.669073,-0.601413
50%,-0.009375,0.000355,0.025975,0.027492
75%,0.662285,0.665312,0.647199,0.710819
max,2.892183,3.000000,2.897228,3.000000


In [76]:
dframe[np.abs(dframe) > 3] = np.nan
dframe.dropna(inplace=True)  
dframe.describe() 

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.017682,0.023077,0.004315,0.066915
std,0.984102,0.952870,0.971865,0.999329
min,-2.584098,-3.000000,-2.978282,-3.000000
25%,-0.656768,-0.655624,-0.669073,-0.601413
50%,-0.009375,0.000355,0.025975,0.027492
75%,0.662285,0.665312,0.647199,0.710819
max,2.892183,3.000000,2.897228,3.000000


# Permutation

#### Sampling without replacement

In [77]:
dframe = DataFrame(np.arange(16).reshape(4, 4)) 
dframe

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [83]:
blender = np.random.permutation(4) 
blender

array([1, 2, 0, 3])

In [84]:
dframe.take(blender) 

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3
3,12,13,14,15


#### Sampling with replacement

In [86]:
box = np.array(["Red", "white", "Black"])
shaker = np.random.randint(0, len(box), size=10)
shaker

array([2, 0, 2, 0, 0, 2, 1, 1, 1, 1])

In [87]:
hand_grabs = box.take(shaker)
hand_grabs 

array(['Black', 'Red', 'Black', 'Red', 'Red', 'Black', 'white', 'white',
       'white', 'white'], 
      dtype='|S5')